РАССТОЯНИЕ ЛЕВЕНШТЕЙНА И КЛАСТЕРИЗАЦИЯ

In [211]:
import csv
import matplotlib.pyplot as plt
import math
import random
import string
import pandas as pd
from collections import OrderedDict

class clustering_Lev_kmeans:
    
    
    def read_data(filename):
        with open(filename, 'rt') as csvfile:
            data = list(csv.reader(csvfile))
            data.pop(0)
            return data
    
    def max_dim(data, use_chars=False):
        max = 0 
        if use_chars:
            for dataset in data:
                if len(dataset[0]) > max:
                    max = len(dataset[0])
        else:
            for dataset in data:
                if len(dataset) > max:
                    max = len(dataset)
        return max
    
    def convert_with_dictionary(data, dictionary):
        for dataset in data:
            for i in range(len(dataset)):
                dataset[i] = dictionary[dataset[i]]
    
    def get_unique_words(data):
        unique_words = set()    
        for datasets in data:
            for datapoint in datasets:
                unique_words.add(datapoint)
    
        return list(unique_words)    

    def create_dicts(data, use_chars=False):   
        unique_words = get_unique_words(data)
        if use_chars == True:
            datapoint2char = {}
            char2datapoint = {}
            char = string.ascii_lowercase[0]
            for datapoint in unique_words:
                datapoint2char[datapoint] = char
                char2datapoint[char] = datapoint
                char = chr(ord(char) + 1) 
            return datapoint2char, char2datapoint
        else:
            datapoint2num = {}
            num2datapoint = {}
            num = 5
            for datapoint in unique_words:
                datapoint2num[datapoint] = num
                num2datapoint[num] = datapoint
                num = num+1
            return datapoint2num, num2datapoint
    
    def join_chars(data):
        for i in range(len(data)):
            data[i] = [''.join(data[i])]
        return data
    
    def split_chars(data):
        for i in range(len(data)):
            data[i] = list(data[i])
        return data
    

    def plot_data(data):    
        index = 0
        for dataset in data:
            plt.plot(dataset)
            index = index+1
        plt.legend()
        plt.show()

    def euclidean_distance(a, b):
        dist = 0
        if len(a) < len(b):
            a,b = b,a
        for i in range(len(a)):
            if i < len(b):
                dist = dist + (a[i] - b[i]) ** 2
            else:
                dist = dist + a[i]**2
        return math.sqrt(dist)
    
    def levenshtein_on_numbers(dataset1, dataset2):
        datapoint2char, char2datapoint = create_dicts([dataset1, dataset2], use_chars=True)        
        convert_with_dictionary([dataset1], datapoint2char)
        convert_with_dictionary([dataset2], datapoint2char)
        join_chars([dataset1])
        join_chars([dataset2])
        distance = d_levenshtein_distance(dataset1, dataset2) 
        split_chars([dataset1])
        split_chars([dataset2]) 
        convert_with_dictionary([dataset1], char2datapoint)
        convert_with_dictionary([dataset2], char2datapoint)  
        return distance
               
    def d_levenshtein_distance(str1, str2):
        d = {}
        for i in range(len(str1) + 1):
            d[(i,0)] = i
        for j in range(len(str2) + 1):
            d[(0,j)] = j
        for i in range(1, len(str1) + 1):
            for j in range(1, len(str2) + 1):
                if str1[i-1] == str2[j-1]:
                    subst_or_equal = d[(i-1, j-1)] + 0
                else:
                    subst_or_equal = d[(i-1, j-1)] + 1
                deletion = d[(i-1,j)] + 1
                insertion = d[(i,j-1)] + 1
                if (i >= 2 and j >= 2) and (str1[i-1] == str2[j-2] and str1[i-2] == str2[j-1]):
                    switch = d[(i-2,j-2)] + 1
                    d[(i,j)] = min(subst_or_equal, deletion, insertion, switch)
                else:
                    d[(i,j)] = min(subst_or_equal, deletion, insertion)
        return d[(len(str1), len(str2))]
    
    def dtw_distance(dataset1, dataset2):
        dtw = {}
        for i in range(len(dataset1)):
            dtw[(i,-1)] = float('inf')
        for i in range(len(dataset2)):
            dtw[(-1,i)] = float('inf')
        dtw[(-1,-1)] = 0
        for i in range(len(dataset1)):
            for j in range(len(dataset2)):
                dist = (dataset1[i] - dataset2[j])**2
                dtw[(i,j)] = dist + min(dtw[(i-1,j)], dtw[(i,j-1)], dtw[(i-1,j-1)]) 
        return math.sqrt(dtw[len(dataset1)-1, len(dataset2)-1])
    
    def k_means(k, data, dist_fun):
        centroids = []
        old_centroids = []
        cluster_for_dataset = []
        clusters = [[] for i in range(k)]
        delta_centroid_sum = 0
        dataset_dim = max_dim(data)
        min_value = 2
        c = list([datapoint for dataset in data for datapoint in dataset])
        c.sort()
        max_value =  c[-1]
        zeros = [0 for i in range(dataset_dim)]
        for cluster in range(k):
            randoms = [random.randint(min_value, max_value) for i in range(dataset_dim)]
            old_centroids.append(zeros)
            centroids.append(randoms)
            delta_centroid_sum = delta_centroid_sum + dist_fun(zeros, randoms)
        while delta_centroid_sum != 0:
            for dataset in data:
                cluster_distances = []
                for cluster in range(k):
                    cluster_distances.append(dist_fun(dataset, centroids[cluster]))
                cluster_for_dataset.append(cluster_distances.index(min(cluster_distances)))
            delta_centroid_sum = 0
            for cluster in range(k):
                cluster_members = []
                for i in range(len(data)):
                    if cluster == cluster_for_dataset[i]:
                        cluster_members.append(data[i])
                old_centroids[cluster] = centroids[cluster]
                datapoint_means = [0 for i in range(dataset_dim)]
                cluster_member_count = len(cluster_members)
                for dataset in cluster_members:
                    for i in range(len(dataset)):
                        datapoint_means[i] = datapoint_means[i] + dataset[i]/cluster_member_count
                centroids[cluster] = datapoint_means
                clusters[cluster] = cluster_members
                delta_centroid_sum = delta_centroid_sum + dist_fun(old_centroids[cluster], centroids[cluster])
        return clusters, centroids

ОЦЕНКА КЛАСТЕРИЗАЦИИ

In [212]:
df = pd.read_csv('C:/Users/alena/Desktop/Data_Scientist/Z08/johnsnowlabs-food-nutrition-value-database/data_new.csv')
df4 = df[['Desk3','cluster']]
df4['cluster'] = df4['cluster'].str.split(' ').str[1]
df4 = df4.sort_values(['cluster'])
df4 = df4.drop_duplicates()

i=0  
sp2_ =[]
for i in range(28):
    str2_ = []
    cd = df4[df4['cluster'] == str(int(i)+1)] 
    j = 0
    for j in range(len(cd)):
        str2_.append(str(cd.iloc[j]['Desk3']))
    sp2_.append(str(str2_)[1:-1])

C:\Users\alena\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [225]:
df = pd.read_csv('C:/Users/alena/Desktop/Data_Scientist/Z08/johnsnowlabs-food-nutrition-value-database/data_new.csv')
df = df.sort_values(['Desk3'])
ls_Names = df["Desk3"].tolist()
ls_Names = df["Desk3"]
ls_Names = list(OrderedDict.fromkeys(ls_Names))
cc = pd.DataFrame(ls_Names)
List = cc.values.tolist()
List
sp_ =[]
str_ = []
cl = clustering_Lev_kmeans
def main():
    data= List
    
    datapoint2num, num2datapoint = cl.create_dicts(data)
    cl.convert_with_dictionary(data, datapoint2num)
    clusters, centroids = cl.k_means(28, data, dtw_distance)
    for i in range(28):
        str_ = []
        cl.convert_with_dictionary(clusters[i], num2datapoint)
        for j in range(len(clusters[i])):
            str_.append(str(clusters[i][j])[2:-2])
        sp_.append(str(str_)[1:-1])

main()

In [226]:
ID = range(28)
dd = {'user_id':ID, 'cluster_id_x':sp2_, 'cluster_id_y':sp_}
dd2 = pd.DataFrame(dd)
dd2


,user_id,cluster_id_x,cluster_id_y
0,0,"'пончики', 'открытый пирог', 'панировочные сух...","'андреа', 'апельсиновый сок', 'батончик из рис..."
1,1,"'грибы белые', 'грибы устрицы', 'гриб', 'грибы...","'арбуз', 'бобер', 'бобы', 'вино', 'гвоздика', ..."
2,2,"'чипсы', 'перекус', 'хумус', 'тамалес', 'чизер...","'агава', 'груша', 'датское тесто', 'желтохвост..."
3,3,"'сливки', 'обезжиренный йогурт', 'сметана', 'ф...","'абиюч', 'авокадо', 'базилик', 'бальзам груша ..."
4,4,"'эдамам', 'шелковица', 'хлопья', 'ячмень', 'яч...","'апельсиново-грейпфрутный сок', 'гуанабанский ..."
5,5,"'макароны', 'лазанья', 'рисовая лапша', 'кунжу...","'бамбуковые побеги', 'банановые чипсы', 'виног..."
6,6,"'маргарин', 'фруктовые масла', 'китовий жир', ...","'болонья', 'грвы', 'каши', 'листья голени', 'м..."
7,7,"'молоко шоколадное', 'молочные шейки', 'молоко...","'выпечка для тостов', 'душистый перец', 'жир и..."
8,8,"'мороженое ванильное', 'лед', 'мороженое клубн...","'бедро индейки', 'брокколи', 'вокас', 'восточн..."
9,9,"'говядина', 'телятина филейка', 'свиная колбас...","'агар', 'английский маффин', 'буррито', 'взбит..."


In [379]:
import pandas as pd
import numpy as np

def BCubedMetrics(df):
    '''
    Функция нужна для расчета BCubed метрик для кластеризации.
    Вдохновение взято отсюда: https://habr.com/ru/company/yandex/blog/500742/
    Основополагающая статья: http://nlp.uned.es/docs/amigo2007a.pdf

    Функция принимает на вход pd.DataFrame(), состоящий из трех столбцов: id, cluster_id_true, cluster_id_computed
    Названя столбцов не имеют значения, но важен их порядок

    Также важно, что один id принадлежит одному кластеру
    '''

    # Переименуем колонки, чтобы проще было в дальнейшем к ним обращаться
    df.columns = ['user_id', 'cluster_id_x', 'cluster_id_y']

    # Найдем размеры кластеров
    true_cluster_size = df.loc[:, ['user_id', 'cluster_id_x']].groupby('cluster_id_x').count().reset_index(). \
        rename(columns={'user_id': 'cluster_id_x_size'})
    computed_cluster_size = df.loc[:, ['user_id', 'cluster_id_y']].groupby('cluster_id_y').count().reset_index(). \
        rename(columns={'user_id': 'cluster_id_y_size'})

    # Присоединим полученные размеры к изначальным данным
    df = pd.merge(df, true_cluster_size, how='left', on='cluster_id_x')
    df = pd.merge(df, computed_cluster_size, how='left', on='cluster_id_y')

    # Для каждого кластера составим лист id, входящих в него
    true_clusters_elements = df.loc[:, ['user_id', 'cluster_id_x']].groupby('cluster_id_x')['user_id']. \
        apply(list).reset_index().rename(columns={'user_id': 'cluster_id_x_elements'})
    computed_clusters_elements = df.loc[:, ['user_id', 'cluster_id_y']].groupby('cluster_id_y')['user_id']. \
        apply(list).reset_index().rename(columns={'user_id': 'cluster_id_y_elements'})

    # Присоединим
    df = pd.merge(df, true_clusters_elements, how='left', on='cluster_id_x')
    df = pd.merge(df, computed_clusters_elements, how='left', on='cluster_id_y')

    # А теперь посчитаем поэлементную длину пересечения кластеров
    df = pd.concat([df, df.apply(lambda x: len(set(x['cluster_id_x_elements']) & set(x['cluster_id_y_elements'])),
                                 axis=1)],
                   axis=1)

    # Переименуем эту колонку
    df.rename(columns={0: 'intersection_size'}, inplace=True)

    # Посчитаем поэлементные метрики
    df['BCP'] = df['intersection_size'] / df['cluster_id_y_size']
    df['BCR'] = df['intersection_size'] / df['cluster_id_x_size']

    # Посчитаем итоговые метрики
    result = np.mean(df[['cluster_id_x', 'BCP', 'BCR']].groupby('cluster_id_x').mean().reset_index()[['BCP', 'BCR']])
    result['BCF1'] = (2 * result['BCP'] * result['BCR']) / (result['BCP'] + result['BCR'])

    return result

In [373]:
BCubedMetrics(dd2)

BCP     1.0
BCR     1.0
BCF1    1.0
dtype: float64

МОДЕЛЬ БЛИЖАЙШЕГО СОСЕДА

In [6]:
from warnings import simplefilter
import pandas as pandas
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB
from sklearn import metrics, datasets
from sklearn.metrics import confusion_matrix
import operator
import numpy as np
import scipy.optimize as optimize
import math

def euclidianDistance(instance1, instance2, length):
    distance = 0
    for x in range(length):
        distance = distance + pow((instance1[x] - instance2[x]), 2)
    return math.sqrt(distance)

def getNeighborsKnn(trainSet, instance, k):
    distances = []
    length = len(trainSet[0]) - 1
    for x in range(len(trainSet)):
        dist = euclidianDistance(instance, trainSet[x], length)
        distances.append((trainSet[x], dist))
    distances.sort(key=operator.itemgetter(1))
    neighbors = []
    for x in range(k):
        neighbors.append(distances[x][0])
    return neighbors

def getDistances(neighbors, y):
    distances = []
    for x in neighbors:
        distance = euclidianDistance(y, x, len(neighbors[0]))
        distances.append(distance)
    return distances

def getRelativeDistances(distances):
    relativeDistances = []
    for x in distances:
        if min(distances) == 0:
            relativeDistances.append(0)
        else:
            relativeDistance = (x - min(distances)) / min(distances)
            relativeDistances.append(relativeDistance)
    return relativeDistances

def getDistancePenalizingFactors(relativeDistances, n):
    penalizingFactors = []
    for x in relativeDistances:
        penalizingFactor = math.exp((-n) * x)
        penalizingFactors.append(penalizingFactor)
    return penalizingFactors

def minimization(resultPredicted, trueResult, penalizingFactors, n_classes):

    def f(x):
        soma = 0
        for i in range(len(resultPredicted)):
            bt = np.reshape(x, (n_classes,n_classes)).transpose()
            rp = np.reshape(np.array(resultPredicted[i]), (n_classes, 1))
            tr = np.reshape(np.array(trueResult[i]), (n_classes, 1))
            soma = soma + (penalizingFactors[i]*(distance.euclidean(np.dot(bt, rp), tr)))
        return soma

    initial_guess = np.array(np.identity(n_classes))
    dimension = initial_guess.shape
    bounds = [(0, 1)]*(n_classes*n_classes)

    eq_cons = ({'type': 'eq',
               'fun': lambda x: np.array([x[0] + x[1] + x[2] - 1,
                                          x[3] + x[4] + x[5] - 1,
                                          x[6] + x[7] + x[8] - 1])})

    result = optimize.minimize(f, initial_guess, method='slsqp', constraints=eq_cons, bounds=bounds)
    return (np.reshape(np.array(result.x), (n_classes,n_classes)))

def correctClassification(b, y, u):
    return (y*(np.dot((np.reshape(b, (len(u[0]),len(u[0]))).transpose()),(u.transpose())))) + ((1 - y)*(u.transpose()))

def main():
    graphResultBefore = []
    graphResultAfter = []
    simplefilter(action='ignore', category=FutureWarning)
           
    data = pandas.read_csv('C:/Users/alena/Desktop/Data_Scientist/Z08/johnsnowlabs-food-nutrition-value-database/data_new.csv')
   
    data = data.drop('Short_Description_ru', axis=1)
    data = data.drop('Short_Description', axis=1)
    data = data.drop('Gram_Weight_Descritpion_1', axis=1)
    data = data.drop('Gram_Weight_Description_2', axis=1)
    
    
    data2 = data.loc[data['Desk3'] == key] 
    classter = data2.iloc[0]['cluster']
    
    data = data.loc[data['cluster'] == classter] 
    data = data.drop('cluster', axis=1)
     
    X_train = data.loc[data['Desk3'] != key] 
    X_train = X_train.drop('Desk3', axis=1)
    
    X_test = data.loc[data['Desk3'] == key] 
    X_test = X_test.drop('Desk3', axis=1)
    
    y_train = data.loc[data['Desk3'] != key] 
    y_train = y_train['Desk3']
    
    y_test = data.loc[data['Desk3'] == key] 
    y_test = y_test['Desk3']
    
    cl = data['Desk3']
    cl = cl.drop_duplicates()
    l_cl = len(cl)
    classes = list(range(1, l_cl +1 ))

    yParameter = [0.1]
    for i in yParameter:
        classifier = GaussianNB()
        classifier.fit(X_train, y_train)
        pred = classifier.predict(X_test)

        accuracyBefore = metrics.accuracy_score(y_test, pred)
        graphResultBefore.append(accuracyBefore)

        matBefore = confusion_matrix(pred, y_test)

        k = 5
        counter = 0
        nParameter = 5
        resultCorrected = []
        for a in range(len(X_test)):
            neighbors = getNeighborsKnn(X_train.values, X_test.values[a], k)
            distances = getDistances(neighbors, X_test.values[a])
            relativeDistances = getRelativeDistances(distances)
            penalizingFactors = getDistancePenalizingFactors(relativeDistances, nParameter)
            predNeighborsVector = classifier.predict_proba(neighbors)

            trueResult = []
            for v in range(len(neighbors)):
                for l in range(len(X_train.values)):
                    if all(neighbors[v] == X_train.values[l]):
                        trueResult.append(y_train.values[l])

            trueResultVector = []
            for true in trueResult:
                aux = []
                if true == classes[0]: 
                    aux = np.zeros(len(classes))
                    aux[0] = 1.0
                if true == classes[1]: 
                    aux = np.zeros(len(classes))
                    aux[1] = 1.0
                if true == classes[2]: 
                    aux = np.zeros(len(classes))
                    aux[2] = 1.0
                trueResultVector.append(aux)
            u = classifier.predict_proba([X_test.values[a]])
            uClass = classifier.predict([X_test.values[a]])

            
    print(uClass)
 

In [12]:
key = 'творог'
main()

['сливки']


ЗАПИСЬ АНАЛОГОВ ИЗ МОДЕЛИ В ФАЙЛ

In [ ]:
data = pandas.read_csv('C:/Users/alena/Desktop/Data_Scientist/Z08/johnsnowlabs-food-nutrition-value-database/data_new.csv')
data = data["Desk3"]
data = data.drop_duplicates()
ll = len(data)
ID = range(len(data))
dd_ = {'ID':ID, 'product':data, 'similar':ID}
dd2_ = pandas.DataFrame(dd_)
dd2_ = dd2_.set_index('ID')
dd2_

for i in ID:
    key = dd2_.loc[i,'product']
    uClass = main()
    dd2_.loc[i,'similar']  = uClass
    
dd2_.to_csv('C:/Users/alena/Desktop/Data_Scientist/Z08/johnsnowlabs-food-nutrition-value-database/similar.csv', index=False)  
    

ПОИСК ПОХОЖЕГО ПРОДУКТА ИЗ ФАЙЛА

In [ ]:
import pandas as pd
data = pd.read_csv('C:/Users/alena/Desktop/Data_Scientist/Z08/johnsnowlabs-food-nutrition-value-database/similar.csv')
key = input("Введите продукт: ")

data2 = data.loc[data['product'] == key]
if len(data2) > 0:
    sim = data2.iloc[0]['similar']
    print("Похожий продукт: ", sim)
if len(data2) == 0:
    data3 = data.loc[data['product'].str.contains(key)]
    bn = data3['product']
    if len(bn) == 0:
        key = key[:3]
        data5 = data.loc[data['product'].str.contains(key)]
        bn5 = data5['product']
        if len(bn5) > 0:
            print ("Возможно вы имели ввиду: ")
            print (" ")
            for i in range(len(bn5)):
                print(bn5.iloc[i])
            prod = input("Введите продукт из списка : ")
            data4 = data.loc[data['product'] == prod]
            if len(data4) > 0:
                sim = data4.iloc[0]['similar']
                print("Похожий продукт: ", sim)
            else:
                print ("Не знаю такой продукт :( ")
    if len(bn) > 0:
        print ("Возможно вы имели ввиду: ")
        print (" ")
        for i in range(len(bn)):
            print(bn.iloc[i])
        prod = input("Введите продукт из списка : ")
        data4 = data.loc[data['product'] == prod]
        if len(data4) > 0:
            sim = data4.iloc[0]['similar']
            print("Похожий продукт: ", sim)
        else:
            print ("Не знаю такой продукт :( ")
    else: ("Не знаю такой продукт :( ")

In [13]:
import pandas as pd
data = pd.read_csv('C:/Users/alena/Desktop/Data_Scientist/Z08/johnsnowlabs-food-nutrition-value-database/similar22.csv',sep =';')
data

,product,similar
0,завтрак,закуска
1,абиюч,замороженные новинки
2,абрикос,чернослив
3,абрикосовый нектар,апельсиновый сок
4,авокадо,перец
...,...,...
1091,яхтверст,обед
1092,ячменная мука,ячмень
1093,ячменный солод,пшеница
1094,ячмень,пшеница


In [14]:
data.to_csv('C:/Users/alena/Desktop/Data_Scientist/Z08/johnsnowlabs-food-nutrition-value-database/similar.csv', index=False)  
    